In [1]:
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import requests
from user_agent import generate_user_agent
from tqdm import tqdm
import matplotlib.pyplot as plt

path = os.getcwd().replace('eurostat_evol', '')
data = pd.read_excel(path + '/data/db_main.xlsx')

# Titre
names = list(data.loc[0])
data = data.loc[1:]
data.columns = names

for col in data.columns:
    try:
        data[col] = data[col].apply(lambda x: 0 if x == '..' else int(x))
    except Exception:
        pass
data = data.drop(['Sort\norder', 'Notes', 'Code', 'Type of data (a)', 'Total', 'Other North', 'Other South'], axis=1)


In [2]:
data = data.set_index(['Major area, region, country or area of destination'])
data = data.drop(['WORLD', 'High-income countries', 'More developed regions',
              'EUROPE', 'Less developed regions', 'Western Europe',
              'Less developed regions, excluding least developed countries',
              'Southern Europe', 'NORTHERN AMERICA', 'AFRICA',
              'Northern Europe', 'Sub-Saharan Africa', 'Middle-income countries',
              'LATIN AMERICA AND THE CARIBBEAN', 'Upper-middle-income countries',
              'ASIA', 'Eastern Africa', 'Caribbean', 'OCEANIA', 'Western Asia',
              'Lower-middle-income countries', 'Least developed countries',
              'Low-income countries', 'South America', 'Northern Africa',
              'Eastern Europe', 'Middle Africa', 'Western Africa',
              'Western Sahara', 'Southern Africa', 'South-Eastern Asia'],
              axis=0)
data = data.reset_index()

In [3]:
data.head()

,"Major area, region, country or area of destination",Year,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,Burundi,1990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Comoros,1990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Djibouti,1990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,289,0,0
3,Eritrea,1990,0,0,0,0,0,247,0,0,...,0,0,0,0,0,0,0,0,91,82
4,Ethiopia,1990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,68,0,0


In [4]:
# All countries
df = []

for i in list(data.index):
    temp = data.loc[i]
    year_list = int(temp['Year'])
    city_list = list(temp.index)[2:]
    figures_list = list(temp)[2:]
    for j in range(len(city_list)):
        df.append([year_list, temp['Major area, region, country or area of destination'],
                       city_list[j], figures_list[j]])

df = pd.DataFrame(df)
df.columns = ['Year', 'destination', 'origin', 'number']

In [5]:
# List number to france
df = df[df['destination'] == 'France']
df = df[['Year', 'origin', 'number']]

In [6]:
# 10 first
list_contry = ['Algeria', 'Morocco', 'Portugal', 'Tunisia', 'Italy', 'Spain',
               'Turkey', 'Germany', 'United Kingdom', 'Belgium']
df['main_cntry'] = df['origin'].apply(lambda x: x if x in list_contry else 'other')
df = df.groupby(['Year', 'main_cntry'], as_index=False)['number'].sum()

In [7]:
df_res = pd.DataFrame()
for year in np.unique(df['Year']):
    df_temp = df[df['Year'] == year]
    nb_tot_fr = np.sum(df_temp['number'])
    df_temp['share'] = (df_temp['number']/nb_tot_fr)*100
    if df_res.empty:
        df_res = df_temp
    else:
        df_res = pd.concat([df_res, df_temp])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [33]:
df_res.to_csv(path + '/data/test.csv')

In [30]:
# Plot
def plot_year(year):
    df_res_year = df_res[df_res['Year'] == year]
    df_res_year = df_res_year.sort_values(by='share', ascending=True)
    N = np.arange(len(df_res_year['main_cntry']))
    plt.barh(N, df_res_year['share'], align='center', alpha=0.5)
    plt.yticks(N, df_res_year['main_cntry'])
    plt.xlabel("Pourcentage d'immigrés en France")
    plt.ylabel("Pays d'origine")

    plt.show()

In [31]:
from __future__ import print_function
from ipywidgets import interact
import ipywidgets as widgets

interact(plot_year, year=widgets.SelectionSlider(
    options=np.unique(df_res['Year']),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
))

interactive(children=(SelectionSlider(continuous_update=False, description='year', options=(1990, 1995, 2000, …

<function __main__.plot_year(year)>

In [32]:
from IPython.display import IFrame
IFrame(src= "https://dash-simple-apps.plotly.host/dash-barplot/", width="100%", height="650px", frameBorder="0")